In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! pip install yelpapi

In [ ]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import string
import spacy
import nltk
from textblob import TextBlob
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import requests
import json
import time
from yelpapi import YelpAPI
import os

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
sp = spacy.load('en_core_web_sm')
all_stopwords = sp.Defaults.stop_words

In [ ]:
name = 'starbucks'

In [ ]:
def get_redirect(id, verbose = False):
  response = requests.get('https://www.yelp.com/biz/{}'.format(id))
  if response.history:
      if verbose:
        print("Request was redirected")

      resp = response.history[-1]
      if verbose:
        print(type(resp))
        print(resp.status_code, resp.url)
        print("Final destination:")
        print(response.status_code, response.url)
  else:
      if verbose:
        print("Request was not redirected")
  return response.url

In [ ]:
os.getcwd()

'/content'

In [ ]:
os.chdir('drive/My Drive/1001')

In [ ]:
baseline = pd.read_csv('DOHMH_New_York_City_Restaurant_Inspection_Results.csv')

In [ ]:
baseline.head(1)

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,50086566,F & M CAMPO BELLO II RESTAURANT,Brooklyn,9128,5 AVENUE,11209.0,7186807888,Spanish,09/16/2019,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,N,26.0,NaN,NaN,11/01/2020,Cycle Inspection / Initial Inspection,40.618348,-74.028921,310.0,43.0,5800.0,3154769.0,3.060870e+09,BK31


In [ ]:
baseline.columns

Index(['CAMIS', 'DBA', 'BORO', 'BUILDING', 'STREET', 'ZIPCODE', 'PHONE',
       'CUISINE DESCRIPTION', 'INSPECTION DATE', 'ACTION', 'VIOLATION CODE',
       'VIOLATION DESCRIPTION', 'CRITICAL FLAG', 'SCORE', 'GRADE',
       'GRADE DATE', 'RECORD DATE', 'INSPECTION TYPE', 'Latitude', 'Longitude',
       'Community Board', 'Council District', 'Census Tract', 'BIN', 'BBL',
       'NTA'],
      dtype='object')

In [ ]:
req_cols = ['CAMIS', 'DBA', 'BUILDING', 'STREET', 'ZIPCODE', 'Latitude', 'Longitude']

In [ ]:
baseline_reduced = baseline[req_cols]
baseline_reduced.drop_duplicates(inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
baseline_reduced.fillna('0', inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [ ]:
def get_business_id(data):
  df_temp = pd.DataFrame(columns={'CAMIS', 'business_id'})
  yelp_api = YelpAPI('UK5xwczHhivsFYK7t-pIBqID4nMjoNpEL4AzB1s9pPkKMNd-T4RTy0xwjKcxVayWF0cU6zd3jjF3hNVyq-3Vpg4sy99loV9I_8dcuywnxjbnqbAHqxvQeohwTpCsX3Yx')
  i = 0
  t = time.time()
  for row in data.itertuples():
    # print(row)
    try:
      res = yelp_api.business_match_query(name= row.DBA, 
                                          state='NY', 
                                          city='New York', 
                                          address1= str(str(row.BUILDING) + ' ' + str(row.STREET)), 
                                          country='US', 
                                          latitude= float(row.Latitude), 
                                          longitude= float(row.Longitude), 
                                          match_threshold='none')['businesses']
    except:
      print(row.CAMIS)
      pass
    if res:
      id = res[0]['id']
    else:
      id = '#'

    data = {'CAMIS': row.CAMIS,
            'business_id': id
    }
    # print(i, data)
    if i % 100 == 0:
      print(i, t - time.time())
    i += 1
    df_temp = df_temp.append(data, ignore_index=True)
  return df_temp


In [ ]:
camis_id_mapping = get_business_id(data = baseline_reduced)

0 -0.5981333255767822
50038524
50056622
50016817
50079781
40685819
50036900
41686200
1000 -588.408833026886
50054234


In [ ]:
yelp_api = YelpAPI('UK5xwczHhivsFYK7t-pIBqID4nMjoNpEL4AzB1s9pPkKMNd-T4RTy0xwjKcxVayWF0cU6zd3jjF3hNVyq-3Vpg4sy99loV9I_8dcuywnxjbnqbAHqxvQeohwTpCsX3Yx')
res = yelp_api.business_match_query(name= 't', 
                                          state='NY', 
                                          city='New York', 
                                          address1= '9128	5 AVENUE', 
                                          country='US', 
                                          latitude= 0, 
                                          longitude= 0, 
                                          match_threshold='none')['businesses']

In [ ]:
res[0]['id']

'tBPbg9RuXPOPbdxyGitZbw'

In [ ]:
df_violations = pd.read_csv('baseline.csv')

FileNotFoundError: ignored

In [ ]:
mapping = pd.read_csv('mapping.csv')

In [ ]:
mapping.head(1)

,Unnamed: 0,CAMIS,business_id
0,0,50086566,tBPbg9RuXPOPbdxyGitZbw


In [ ]:
df = pd.read_csv('scraped.csv')
df.head()

,business_id,rating,review,date
0,tBPbg9RuXPOPbdxyGitZbw,5,Food is always very good. Perfect place to pic...,11/1/2020
1,tBPbg9RuXPOPbdxyGitZbw,1,Disgusting food taste horrible and the women a...,10/30/2020
2,tBPbg9RuXPOPbdxyGitZbw,3,"This is a buffet type style spot, meaning the ...",9/23/2020
3,tBPbg9RuXPOPbdxyGitZbw,1,"The guy with the hat, sold me burnt plantains!...",5/31/2020
4,tBPbg9RuXPOPbdxyGitZbw,4,"Very solid Dominican food, especially for Bay ...",5/17/2020


In [ ]:
# df.drop(columns= ['Unnamed: 0'], inplace=True)
# df.head()

,business_id,rating,review,date
0,tBPbg9RuXPOPbdxyGitZbw,5,Food is always very good. Perfect place to pic...,11/1/2020
1,tBPbg9RuXPOPbdxyGitZbw,1,Disgusting food taste horrible and the women a...,10/30/2020
2,tBPbg9RuXPOPbdxyGitZbw,3,"This is a buffet type style spot, meaning the ...",9/23/2020
3,tBPbg9RuXPOPbdxyGitZbw,1,"The guy with the hat, sold me burnt plantains!...",5/31/2020
4,tBPbg9RuXPOPbdxyGitZbw,4,"Very solid Dominican food, especially for Bay ...",5/17/2020


In [ ]:
# df = pd.DataFrame(columns={'date', 'review', 'rating', 'business_id'})
df = pd.read_csv('scraped.csv')
# if not df:
#     df = pd.DataFrame(columns={'date', 'review', 'rating', 'business_id'})
ids_scraped = df['business_id'].unique()
id = 'bl3-rGjqjaJa_nkW4aMlIg'
name = 'starbucks'
err = []
for row in mapping.itertuples():
    i = 0
    print(row)
    if row.business_id in ids_scraped:
        print('Already scraped')
        continue
    while True:
        # get webpage
        # url = 'https://www.yelp.com/biz/bl3-rGjqjaJa_nkW4aMlIg&start={}&sort_by=date_desc'.format(i)
        url = get_redirect(row.business_id, verbose = False)
        query = '?start={}&sort_by=date_desc'.format(i)

        url += query
        # print(url)
        flag = False
        try:
            ourUrl = urllib.request.urlopen(url)
        except:
            flag = True
            err.append(row.business_id)
            pass
        if flag:
            break
        soup = BeautifulSoup(ourUrl,'html.parser')
        rows = soup.find_all('div',{'class': 'main-content-wrap main-content-wrap--full'})
        # print(rows[0].text[6:-4])
        rows = json.loads(rows[0].text[6:-4])

        #Increment the page number specifier
        i += 20

        #Break if no rows found
        if not rows['bizDetailsPageProps']['reviewFeedQueryProps']['reviews']:
            break
        df_temp = pd.json_normalize(rows['bizDetailsPageProps']['reviewFeedQueryProps']['reviews'])

        required_columns = ['localizedDate', 'comment.text', 'rating']
        col_names = ['date', 'review', 'rating']

        df_req = df_temp[required_columns]
        df_req.columns = col_names
        
        df_req['business_id'] = row.business_id

        df = df.append(df_req, ignore_index=True)
        time.sleep(8)
    print(df.shape)

Pandas(Index=0, _1=0, CAMIS=50086566, business_id='tBPbg9RuXPOPbdxyGitZbw')
Already scraped
Pandas(Index=1, _1=1, CAMIS=50097893, business_id='43viL4nGzaYU3IvYZpp6BQ')
Already scraped
Pandas(Index=2, _1=2, CAMIS=50018578, business_id='1KGvtMU7VBcdlvxo2brdQg')
Already scraped
Pandas(Index=3, _1=3, CAMIS=50015174, business_id='ZIjGYwImNzMwQOkMg7XGHA')
Already scraped
Pandas(Index=4, _1=4, CAMIS=50074043, business_id='FR_E9_Z3WG5HjeIUyTKSWw')
Already scraped
Pandas(Index=5, _1=5, CAMIS=41429861, business_id='4cP9IJekfdFmx8vpMVoY9Q')
Already scraped
Pandas(Index=6, _1=6, CAMIS=50042128, business_id='zEHzdRjEa23tcH585_SqHw')
Already scraped
Pandas(Index=7, _1=7, CAMIS=41110540, business_id='RgoCXonPjr78JUryZ9KkAQ')
Already scraped
Pandas(Index=8, _1=8, CAMIS=50084763, business_id='SU0_Hd_ET94_maeuIB0_vg')
Already scraped
Pandas(Index=9, _1=9, CAMIS=50012215, business_id='lGLeaVcg7U4tOfJFhVqGng')
Already scraped
Pandas(Index=10, _1=10, CAMIS=50061293, business_id='-_LtKO0FdpINUMAWMfDOOQ')
Al

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(4939, 4)
Pandas(Index=43, _1=43, CAMIS=50059003, business_id='6evDC62OwL1SewmR6ui6lw')
(4996, 4)
Pandas(Index=44, _1=44, CAMIS=41579300, business_id='aidZpXrufp3t-cV1tMTzOQ')
(5173, 4)
Pandas(Index=45, _1=45, CAMIS=41154571, business_id='YCYGtlQ1WmWcpDYYSJKeNQ')
(5181, 4)
Pandas(Index=46, _1=46, CAMIS=50061521, business_id='UB1mGugr1pvgUUP1eTNXrg')
(5181, 4)
Pandas(Index=47, _1=47, CAMIS=50060763, business_id='#')
(5181, 4)
Pandas(Index=48, _1=48, CAMIS=50057988, business_id='CesS-E8Q3ztX6VnkXO6wBg')
(5457, 4)
Pandas(Index=49, _1=49, CAMIS=41099763, business_id='kCk2uqmmJBQ80iLuncNhlQ')
(5538, 4)
Pandas(Index=50, _1=50, CAMIS=50007736, business_id='1qOS6TKzsaiVVeMVbl6sCA')
(6041, 4)
Pandas(Index=51, _1=51, CAMIS=41690737, business_id='1ZstLMeTotRyA8xo8Xa77w')


ConnectionResetError: ignored

In [ ]:
err

['UB1mGugr1pvgUUP1eTNXrg', '#']

In [ ]:
df.shape

(6221, 4)

In [ ]:
df.to_csv('scraped.csv',index=False)

In [ ]:
# sentiment analysis
df['polarity'] = df['review'].apply(lambda x: TextBlob(x).sentiment[0])
df['polarity_no_stopwords'] = df['review_no_stopwords'].apply(lambda x: TextBlob(x).sentiment[0])
df.head()

,date,rating,review,review_no_stopwords,name,polarity,polarity_no_stopwords
0,9/13/2020,1,disinterested staff is bad enough but what i w...,disinterested staff bad served today ordering ...,starbucks,-0.050000,-0.066667
1,8/20/2020,1,i cannot believe iamp39m writing this right no...,believe iamp39m writing right incredibly rude ...,starbucks,0.085268,0.035913
2,8/10/2020,1,let me start off by saying the drinks i ordere...,let start saying drinks ordered tasted amazing...,starbucks,0.186364,0.255556
3,7/24/2020,1,you need to prune some dead branches off this ...,need prune dead branches bush specifically cas...,starbucks,0.083333,-0.022222
4,6/30/2020,1,i come to this store daily i came in at 656pm ...,come store daily came 656pm told store closed ...,starbucks,-0.150000,-0.150000


In [ ]:
df.groupby('rating')['polarity'].mean().reset_index()

,rating,polarity
0,1,-0.039939
1,2,0.128937
2,3,0.114549
3,4,0.225554
4,5,0.372342


It looks like the polarity range is larger when we extract sentiments from reviews with no stop words, so we'll use `polarity_no_stopwods` for our model.

In [ ]:
for i in data.keys():
  if type(data[i]) == dict:
    for j in data[i].keys():
      if type(data[i][j]) == dict:
        if 'reviews' in data[i][j].keys():
          print(j)
          print(i)
          print(data[i].keys())
        # print(data[i][j].keys())

reviewFeedQueryProps
bizDetailsPageProps
dict_keys(['facebookCustomAudienceProps', 'callToActionProps', 'liverampTrackingProps', 'realtorDataProps', 'adUnitsBelowReviewProps', 'showOnTheMenu', 'bizQuestionsProps', 'isStickySidebarEnabled', 'interfaceLocaleCode', 'verifiedLicenseProps', 'rewardsWidgetProps', 'showCollapsedPage', 'surveyQuestionsModalProps', 'adUnitsAboveReviewProps', 'adUnitsSidebarProps', 'showPartnerAttribution', 'claimStatusGQLProps', 'popularDishesCarouselProps', 'checkInOfferProps', 'photoHeaderProps', 'claimReminderProps', 'showGoogleOneTap', 'sponsoredBusinessHighlightsProps', 'specialHoursAlertProps', 'businessPostsCarouselProps', 'serviceOfferingsProps', 'mediaShowcaseProps', 'businessAlertProps', 'claimTeaserSidebarProps', 'yelpGuaranteedProps', 'bizPortfolioProps', 'forceSsr', 'businessIsAdvertiser', 'possibleBusinessOwnerModalProps', 'giftCertificateProps', 'adCarouselAboveReviewProps', 'notRecommendedReviewsProps', 'requestAQuoteStickyBarProps', 'followButt

In [ ]:
#Backup






# df = pd.DataFrame(columns={'name', 'date', 'review', 'rating', 'review_no_stopwords'})
# id = 'bl3-rGjqjaJa_nkW4aMlIg'
# name = 'starbucks'
# i = 0
# while True:
#   # get webpage
#   # url = 'https://www.yelp.com/biz/bl3-rGjqjaJa_nkW4aMlIg&start={}&sort_by=date_desc'.format(i)
#   url = get_redirect(id)
#   query = '?start={}&sort_by=date_desc'.format(i)

#   url += query
#   print(url)
#   ourUrl = urllib.request.urlopen(url)
#   soup = BeautifulSoup(ourUrl,'html.parser')
#   rows = soup.find_all('div',{'class': 'main-content-wrap main-content-wrap--full'})
#   rows = json.loads(rows[0].text[6:-4])

#   #Increment the page number specifier
#   i += 20
  
# #   test = rows['bizDetailsPageProps']['reviewFeedQueryProps']['reviews']
# #   break
#   #Break if no rows found
#   if not rows['bizDetailsPageProps']['reviewFeedQueryProps']['reviews']:
#       break
#   for rev in rows['bizDetailsPageProps']['reviewFeedQueryProps']['reviews']:
#     review = rev['comment']['text']
#     rating = int(rev['rating'])
#     date = rev['localizedDate']

#     review = review.lower()
#     review = "".join(l for l in review if l not in string.punctuation)

#     # remove stopwords
#     review_no_stopwords = word_tokenize(review)
#     review_no_stopwords = " ".join([word for word in review_no_stopwords if not word in all_stopwords])

#     data = {'name': name,
#             'date': date,
#             'rating': rating,
#             'review_no_stopwords': review_no_stopwords,
#             'review': review,
#     }
#     df = df.append(data, ignore_index=True)
  
# #   time.sleep(8)